In [18]:
import pandas as pd

In [19]:
df = pd.read_csv('/content/fraudTrain.csv')
df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0.0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0.0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0.0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0.0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0.0


In [20]:
df.columns

Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')

In [21]:
#dropping unneccessary columns
df.drop(columns = ['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant',
       'first', 'last', 'street', 'city', 'zip',
       'lat', 'long', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long'], inplace=True)

In [22]:
df.head()

,category,amt,gender,state,city_pop,job,is_fraud
0,misc_net,4.97,F,NC,3495,"Psychologist, counselling",0.0
1,grocery_pos,107.23,F,WA,149,Special educational needs teacher,0.0
2,entertainment,220.11,M,ID,4154,Nature conservation officer,0.0
3,gas_transport,45.00,M,MT,1939,Patent attorney,0.0
4,misc_pos,41.96,M,VA,99,Dance movement psychotherapist,0.0


In [23]:
df.isnull().sum()

,0
category,0
amt,0
gender,0
state,0
city_pop,0
job,0
is_fraud,1


In [24]:
df.dropna(inplace=True)

In [25]:
df.drop_duplicates(inplace=True)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397049 entries, 0 to 399325
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   category  397049 non-null  object 
 1   amt       397049 non-null  float64
 2   gender    397049 non-null  object 
 3   state     397049 non-null  object 
 4   city_pop  397049 non-null  int64  
 5   job       397049 non-null  object 
 6   is_fraud  397049 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 24.2+ MB


In [27]:
#splitting the dataframe for training
X = df.drop(columns=['is_fraud'], inplace=False)
y = df['is_fraud']

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_pipeline = Pipeline(
    [('scaler', StandardScaler())]
)
cat_pipeline = Pipeline(
    [('1hot', OneHotEncoder())]
)
num_df = df.drop(columns = ['category','gender', 'state', 'job'] )
cat_df = df.drop(columns = ['city_pop', 'amt'] )

In [29]:
#preprocessing
from sklearn.compose import ColumnTransformer

num_attribs = list(num_df)
cat_attribs = list(cat_df)

pipeline = ColumnTransformer([
    ('num', StandardScaler(), num_attribs ),
    ('cat', OneHotEncoder(), cat_attribs)
], remainder='passthrough')

X = pipeline.fit_transform(df)

In [30]:
y.value_counts()

,count
is_fraud,
0.0,394401
1.0,2648


In [31]:
#Balancing the dataset
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='minority')
X, y = smote.fit_resample(X, y)

In [32]:
y.value_counts()

,count
is_fraud,
0.0,394401
1.0,394401


In [33]:
# train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [45]:
X_train = X_train.toarray()
y_train = y_train.values
X_test = X_test.toarray()
y_test = y_test.values

In [46]:
import tensorflow as tf
import keras

model = keras.models.Sequential([
      # input layer + hidden layer 1
    keras.layers.Dense(300, input_shape = (553,), activation = 'relu'),
    # hidden layer 2
    keras.layers.Dense(150, activation = 'relu'),
    #output layer
    keras.layers.Dense(1, activation = 'sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [47]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 300)                 │         166,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 150)                 │          45,150 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │             151 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 211,501 (826.18 KB)

 Trainable params: 211,501 (826.18 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=200)

Epoch 1/10
3156/3156 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.9979 - loss: 0.0091 - val_accuracy: 1.0000 - val_loss: 3.8225e-07
Epoch 2/10
3156/3156 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 1.0000 - loss: 2.3422e-07 - val_accuracy: 1.0000 - val_loss: 4.3867e-08
Epoch 3/10
3156/3156 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 1.0000 - loss: 2.9774e-08 - val_accuracy: 1.0000 - val_loss: 7.6504e-09
Epoch 4/10
3156/3156 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 1.0000 - loss: 5.4176e-09 - val_accuracy: 1.0000 - val_loss: 1.6539e-09
Epoch 5/10
3156/3156 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 1.0000 - loss: 1.2166e-09 - val_accuracy: 1.0000 - val_loss: 4.4065e-10
Epoch 6/10
3156/3156 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 1.0000 - loss: 3.4115e-10 - val_accuracy: 1.0000 - val_loss: 1.5345e-10
Epoch 7/10
3156/3156 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 1.0000 - loss: 1.2626e-10 - val_accuracy: 1.0000 - val_loss: 7.0586e-11
Epoch 8/10
3156/3156 ━━━━━━━━━━━━━━